In [41]:
!uv pip install linearmodels

Audited 1 package in 3ms


In [11]:
import pandas as pd

In [14]:
df_fin = pd.read_csv("./final_financials_trim.csv")

In [15]:
df_pred = pd.read_csv("./final_predictions.csv")

In [18]:
df_pred

,path,ticker,filing,date,prediction_idx,chain_of_thought,estimated_mw,score,confidence
0,./edgartools-data/ABTS/6-K/2025-03-19.md,ABTS,6-K,2025-03-19.md,1583.0,"""The filing from March 2025 indicates that Abi...",7.0,2.0,85.0
1,./edgartools-data/ABTS/6-K/2025-03-05.md,ABTS,6-K,2025-03-05.md,1353.0,"""I meticulously reviewed the provided Form 6-K...",0.0,0.0,100.0
2,./edgartools-data/ABTS/6-K/2025-01-06.md,ABTS,6-K,2025-01-06.md,330.0,"""The filing is a Form 6-K from January 2025. T...",22.0,3.0,98.0
3,./edgartools-data/ABTS/6-K/2021-12-06.md,ABTS,6-K,2021-12-06.md,775.0,"""The company, Moxian (BVI) Inc., filed a Form ...",11.0,3.0,75.0
4,./edgartools-data/ABTS/6-K/2021-10-12.md,ABTS,6-K,2021-10-12.md,2728.0,"""The provided Form 6-K for MOXIAN (BVI) INC, d...",0.0,0.0,100.0
...,...,...,...,...,...,...,...,...,...
3226,./edgartools-data/NCTY/6-K/2009-11-24.md,NCTY,6-K,2009-11-24.md,1500.0,"""The provided document is a Form 6-K filed by ...",0.0,0.0,100.0
3227,./edgartools-data/NCTY/6-K/2009-08-28.md,NCTY,6-K,2009-08-28.md,2480.0,"""The provided regulatory filing is Form 6-K fo...",0.0,0.0,100.0
3228,./edgartools-data/NCTY/6-K/2009-07-15.md,NCTY,6-K,2009-07-15.md,1712.0,"""The provided Form 6-K is from THE9 LIMITED, f...",0.0,0.0,100.0
3229,./edgartools-data/NCTY/6-K/2009-03-10.md,NCTY,6-K,2009-03-10.md,2700.0,"""The provided Form 6-K filing from March 2009 ...",0.0,0.0,100.0


roa = b0 + b1\*score +b2\*confidence + b2\*filingsVol + b3\*ln(assets) + b4\*leverage + b5\*liquidity + b6\*pBRatio + b7\*ln(btcPrice) + b8\*ln(hashrate) + firmFixedEffects


In [27]:
def get_fiscal_quarter(date_str):
    """Convert date string to fiscal quarter format (YYYYqQ)."""
    date = pd.to_datetime(date_str).to_period("Q")
    return date.strftime("%Yq%q")

In [28]:
df_pred["date"] = pd.to_datetime(
    df_pred["date"].astype("str").str.replace(r"\.md$", "", regex=True)
)
df_pred["quarter"] = df_pred["date"].apply(get_fiscal_quarter)

In [30]:
df_final = df_fin.copy(deep=True)

In [ ]:
df_pred_agg = df_pred.groupby(["ticker", "quarter"]).agg(
    score_max=("score", "max"),
    score_mean=("score", "mean"),
    score_count=("score", "count"),
    confidence_max=("confidence", "max"),
    confidence_mean=("confidence", "mean"),
)

In [45]:
merged_df = pd.merge(df_fin, df_pred_agg, on=["ticker", "quarter"], how="inner")

In [46]:
merged_df

,ticker,quarter,roa,log_assets,leverage,liquidity,pb_ratio,log_btc_price,log_hashrate,score_max,score_mean,score_count,confidence_max,confidence_mean
0,ARLP,2025q3,0.032700,21.790849,0.013213,32.486218,1.554364,11.605403,20.878149,3.0,3.0,1,75.0,75.0
1,ARLP,2025q2,0.046462,21.777385,0.012920,0.019169,1.539672,11.581615,20.630243,1.0,1.0,1,10.0,10.0
2,ARLP,2025q1,0.025487,21.788919,0.012799,0.028013,1.602498,11.375819,20.584988,3.0,1.5,2,100.0,97.5
3,ARLP,2024q4,0.005601,21.793386,0.166956,0.046973,1.612357,11.446106,20.581368,3.0,1.5,2,100.0,85.0
4,ARLP,2024q3,0.028455,21.832545,0.162401,0.064452,1.427064,11.095719,20.339314,3.0,1.5,2,100.0,87.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,NCTY,2017q2,-0.496449,19.523342,1.157006,0.011415,-0.055420,7.840584,15.510817,0.0,0.0,1,95.0,95.0
530,NCTY,2017q1,-0.674327,19.675989,0.968799,0.110798,-0.073927,6.946755,15.219958,0.0,0.0,3,100.0,100.0
531,NCTY,2016q4,-1.591270,19.675989,0.968799,0.110798,-58.481220,6.835550,14.767149,0.0,0.0,1,100.0,100.0
532,NCTY,2015q4,-0.714807,20.103546,0.310185,0.091082,-20.153065,6.071269,13.596915,0.0,0.0,2,100.0,100.0


In [ ]:
merged_df["quarter_ts"] = pd.PeriodIndex(merged_df["quarter"], freq="Q").to_timestamp()
regression_df = merged_df.set_index(["ticker", "quarter_ts"])

In [50]:
regression_df

quarter       roa  log_assets  leverage  liquidity  \
ticker quarter_ts                                                      
ARLP   2025-07-01  2025q3  0.032700   21.790849  0.013213  32.486218   
       2025-04-01  2025q2  0.046462   21.777385  0.012920   0.019169   
       2025-01-01  2025q1  0.025487   21.788919  0.012799   0.028013   
       2024-10-01  2024q4  0.005601   21.793386  0.166956   0.046973   
       2024-07-01  2024q3  0.028455   21.832545  0.162401   0.064452   
...                   ...       ...         ...       ...        ...   
NCTY   2017-04-01  2017q2 -0.496449   19.523342  1.157006   0.011415   
       2017-01-01  2017q1 -0.674327   19.675989  0.968799   0.110798   
       2016-10-01  2016q4 -1.591270   19.675989  0.968799   0.110798   
       2015-10-01  2015q4 -0.714807   20.103546  0.310185   0.091082   
       2014-10-01  2014q4  0.066135   20.064193  0.005917   0.350805   

                    pb_ratio  log_btc_price  log_hashrate  score_max  \
ticker quarter_ts                                                      
ARLP   2025-07-01   1.554364      11.605403     20.878149        3.0   
       2025-04-01   1.539672      11.581615     20.630243        1.0   
       2025-01-01   1.602498      11.375819     20.584988        3.0   
       2024-10-01   1.612357      11.446106     20.581368        3.0   
       2024-07-01   1.427064      11.095719     20.339314        3.0   
...                      ...            ...           ...        ...   
NCTY   2017-04-01  -0.055420       7.840584     15.510817        0.0   
       2017-01-01  -0.073927       6.946755     15.219958        0.0   
       2016-10-01 -58.481220       6.835550     14.767149        0.0   
       2015-10-01 -20.153065       6.071269     13.596915        0.0   
       2014-10-01   0.475355       5.739793     12.654413        0.0   

                   score_mean  score_count  confidence_max  confidence_mean  
ticker quarter_ts                                                            
ARLP   2025-07-01         3.0            1            75.0             75.0  
       2025-04-01         1.0            1            10.0             10.0  
       2025-01-01         1.5            2           100.0             97.5  
       2024-10-01         1.5            2           100.0             85.0  
       2024-07-01         1.5            2           100.0             87.5  
...                       ...          ...             ...              ...  
NCTY   2017-04-01         0.0            1            95.0             95.0  
       2017-01-01         0.0            3           100.0            100.0  
       2016-10-01         0.0            1           100.0            100.0  
       2015-10-01         0.0            2           100.0            100.0  
       2014-10-01         0.0            2           100.0            100.0  

[534 rows x 13 columns]

In [85]:
from linearmodels import PanelOLS

formula = "roa ~ score_mean + score_count + log_assets + leverage + liquidity + pb_ratio + log_btc_price + log_hashrate"

mod = PanelOLS.from_formula(formula, data=regression_df)
res = mod.fit(cov_type="clustered", cluster_entity=True)
print(res)


                          PanelOLS Estimation Summary                           
Dep. Variable:                    roa   R-squared:                        0.6949
Estimator:                   PanelOLS   R-squared (Between):              0.9352
No. Observations:                 534   R-squared (Within):               0.6109
Date:                Thu, Dec 04 2025   R-squared (Overall):              0.6949
Time:                        03:56:23   Log-likelihood                   -954.33
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      149.72
Entities:                          27   P-value                           0.0000
Avg Obs:                       19.778   Distribution:                   F(8,526)
Min Obs:                       1.0000                                           
Max Obs:                       61.000   F-statistic (robust):           2.97e+04
                            